In [1]:
import numpy as np
import torch

# テンソルの表示設定：
# ・edgeitems=2 → 各次元の両端2つずつだけ表示
# ・threshold=50 → 要素数が50を超えると中略表示
torch.set_printoptions(edgeitems=2, threshold=50)

In [2]:
with open("../../data/p1ch4/jane-austen/1342-0.txt", encoding="utf8") as f:
    text = f.read()

In [3]:
lines = text.split("\n")
line = lines[200]
line

'“Impossible, Mr. Bennet, impossible, when I am not acquainted with him'

In [4]:
letter_t = torch.zeros(len(line), 128)  # <1>
letter_t.shape

torch.Size([70, 128])

In [5]:
# 文字列 line を小文字化・前後の空白除去した上で1文字ずつ処理
for i, letter in enumerate(line.lower().strip()):

    # ASCIIコードが128未満（標準ASCII文字）の場合はそのコードを使用、
    # それ以外（非ASCII文字）の場合は0番目のインデックスに集約
    letter_index = ord(letter) if ord(letter) < 128 else 0  # <1>

    # i番目の位置に対応するone-hotベクトルを構築（該当インデックスに1を立てる）
    letter_t[i][letter_index] = 1

In [6]:
def clean_words(input_str):
    # 削除対象の句読点記号を定義
    punctuation = '.,;:"!?”“_-'

    # 入力文字列を小文字化し、改行をスペースに変換し、空白で分割して単語リスト化
    word_list = input_str.lower().replace("\n", " ").split()

    # 各単語から先頭・末尾の句読点を除去
    word_list = [word.strip(punctuation) for word in word_list]

    return word_list


# 1行のテキスト line に対して単語クリーニングを実施
words_in_line = clean_words(line)

# 元の行とクリーニング済み単語リストを確認
line, words_in_line

('“Impossible, Mr. Bennet, impossible, when I am not acquainted with him',
 ['impossible',
  'mr',
  'bennet',
  'impossible',
  'when',
  'i',
  'am',
  'not',
  'acquainted',
  'with',
  'him'])

In [7]:
# 入力テキストから単語を抽出・前処理し、重複を除去してソート
word_list = sorted(set(clean_words(text)))

# 単語リストから辞書を作成：{単語: インデックス}
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

# 単語辞書の語彙数と、"impossible" のインデックスを確認
len(word2index_dict), word2index_dict["impossible"]

(7261, 3394)

In [8]:
# 各単語に対応するone-hotベクトルを格納するテンソルを初期化
# 行：文中の単語数、列：語彙数（ボキャブラリサイズ）
word_t = torch.zeros(len(words_in_line), len(word2index_dict))

# 文中の各単語に対して one-hot ベクトルを生成
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]  # 対応する語彙インデックスを取得
    word_t[i][word_index] = 1  # one-hotベクトルの該当インデックスに1を立てる
    print("{:2} {:4} {}".format(i, word_index, word))  # デバッグ用に表示

# 最終的なone-hotテンソルのサイズを確認
print(word_t.shape)

 0 3394 impossible
 1 4305 mr
 2  813 bennet
 3 3394 impossible
 4 7078 when
 5 3315 i
 6  415 am
 7 4436 not
 8  239 acquainted
 9 7148 with
10 3215 him
torch.Size([11, 7261])
